# MODEL TESTING

In [23]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import string
import pickle
import regex as re

In [29]:
testPos="this movie looks very interesting, i love the fact that the actors do a great job in showing how people lived in the 18th century, which wasn't very good at all. But atleast this movie recreates this scenes! "
testNeg="very good start, but movie started becoming boring at some point and unfortunately i didn't feel like this was properly produced as there was too much background noise, and the actors didn't look motivated at all "

In [30]:
test_data=tf.data.Dataset.from_tensor_slices([[testPos], [testNeg],])

In [31]:
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 250
def standardization(inputData):
    lowerCaseOutput = tf.strings.lower(inputData)
    noTagOutput = tf.strings.regex_replace(lowerCaseOutput, "<[^>]+>", " ")
    noPunctOutput = tf.strings.regex_replace(noTagOutput, "[%s]" % re.escape(string.punctuation), "")
    return noPunctOutput

with open('Vectorizer/vectorizer.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [32]:
vectorizer = tf.keras.layers.TextVectorization(
    standardize=standardization,
    max_tokens=VOCAB_SIZE,
    output_sequence_length=SEQUENCE_LENGTH,
    output_mode='int',
)
vectorizer.set_vocabulary(vocab)

In [33]:
model = tf.keras.models.load_model('Models/convModel(Word2Vec).h5')

In [34]:
def vectorize(review):
    return vectorizer(review)
test_data_vectorized = test_data.map(vectorize)

In [35]:
model.predict(test_data_vectorized)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


array([[0.8984775 ],
       [0.03724153]], dtype=float32)

## Inference Ready Model

In [26]:
inputs = tf.keras.Input(shape=(1, ), dtype=tf.string)
vectorized = vectorizer(inputs)
outputs = model(vectorized)
inferenceReadyModel = tf.keras.Model(inputs, outputs)
inferenceReadyModel.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 250)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convModel (Sequential)          │ (None, 1)              │     3,029,921 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,029,921 (11.56 MB)

 Trainable params: 29,921 (116.88 KB)

 Non-trainable params: 3,000,000 (11.44 MB)

In [27]:
inferenceReadyModel.save('Models/InferenceReadyModel.h5')

In [36]:
inferenceReadyModel.predict(test_data)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


array([[0.8984775 ],
       [0.03724153]], dtype=float32)